<a href="https://colab.research.google.com/github/jackmiller-hash/Steam-review-scraper/blob/master/Steam_Sentiment_Scraper_500Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Setup
!pip install steamreviews textblob pandas
import steamreviews, json, time, os, nltk, pandas as pd
from textblob import TextBlob

for pkg in ['punkt', 'brown', 'wordnet', 'punkt_tab']:
    nltk.download(pkg, quiet=True)

# 2. Configuration
APP_ID = 1903340
OUT_FILE = 'Expedition33_Report.csv'

# We use a very small batch size to force the 15-second pause logic
params = {
    'language': 'english',
    'purchase_type': 'all',
    'max_reviews': 500
}

print(f"--- Downloading reviews for {APP_ID} ---")
# The library will fetch 100 reviews per 'request'
steamreviews.download_reviews_for_app_id(APP_ID, chosen_request_params=params)

# 3. Wait for file to write
time.sleep(5)
path = f'data/review_{APP_ID}.json'

if os.path.exists(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    results = []
    for r_id in data.get('reviews', {}):
        text = data['reviews'][r_id].get('review', '')
        if text:
            score = TextBlob(text).sentiment.polarity
            results.append({'Review': text, 'Score': score})

    df = pd.DataFrame(results)

    if not df.empty:
        # Sort and Extract
        favs = df.sort_values('Score', ascending=False).head(5)
        pains = df.sort_values('Score', ascending=True).head(5)

        print("\nTOP 5 FAN FAVORITES:")
        for _, row in favs.iterrows():
            print(f"- {row['Review'][:150]}...")

        print("\nTOP 5 PAIN POINTS:")
        for _, row in pains.iterrows():
            print(f"- {row['Review'][:150]}...")

        df.to_csv(OUT_FILE, index=False)
        print(f"\nSUCCESS: Created {OUT_FILE}")
    else:
        print("Data found, but no review text extracted.")
else:
    print("Folder 'data' is empty. Steam Cooldown is likely active. Wait 15 mins.")

--- Downloading reviews for 1903340 ---
[appID = 1903340] expected #reviews = 127618
Number of queries 150 reached. Cooldown: 310 seconds
Number of queries 150 reached. Cooldown: 310 seconds
Number of queries 150 reached. Cooldown: 310 seconds
Number of queries 150 reached. Cooldown: 310 seconds
